In [9]:
import torch
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.draw import random_shapes
import cv2
import pandas as pd
import random

In [1]:
numTransforms = 15
trainSteps = 10000
batchSize = 30 #training 
sparsePen = 1
derivPen = .5
epsilon = .01 #variance of normal desntiy in the smoothed bump function.
eta = .01 #learning rate of phi, the dictionary of transforms
eta2 = .05 #learning rate of beta, the transform coeff
eta3 = .1 #Learning rate of center
eta4 = .1 #Learning rate of radii
numInference = 15*10 #number of basis trans * length of movie sequence

In [20]:
#Training algorithm, image sequences with one shape. No need to infer center/radii yet.
def timeDiff(beta):
    #assuming beta has shape bkt, where k for trans, t for length, B for number of images (usually batch size)
    
    Zeroes = np.zeros((beta.shape[0],1,beta.shape[2]))
    timeDeriv = np.diff(beta, axis= 1)
    X = np.concatenate((Zeroes, timeDeriv), axis=1)
    Y = np.concatenate((timeDeriv,Zeroes), axis =1)
    Z = -1*np.sign(X)+np.sign(Y)
    
    return(Z)

def inferBeta(Input, Output, phi, sparsePen, derivPen, eta2):
    #Assume data has shape (B, T, n2), where number of images(usually batch size) , T is time, and n2 is n^2 for nxn image
    #phi is of shape (i,i',k) where i,i' denote image dim, k denotes transform, T is time, and B is batch
    #Input are first T-1 frames. Output are frames 2 to T. Infer i^th frame from previous.
    
    beta = np.zeros((Input.shape[0],Input.shape[1],phi.shape[2])) #initialize beta at 0. beta is shape btk
    
    for step in range(numInference):
        predictions = np.einsum('btk,ijk,btj -> bti', beta, phi, Input)
        error = Output - predictions
        dBeta = np.einsum('bti,ijk,btj -> btk', error, phi, Input) - sparsePen*np.sign(beta) - derivPen*timeDiff(beta)
        beta += eta2*dBeta
        
    return(beta)
    
def learnTransform(Input, Output, phi, beta, eta, batchSize):
    
    predictions = np.einsum('btk,ijk,btj -> bti', beta, phi, Input)
    error = Output - predictions
    dPhi = np.einsum('bti, btj, btk -> ijk', error, Input, beta)
    phi += (eta/batchSize)*dPhi
    
    return(phi)
    

In [12]:
#Traning algo, image sequences with one shape.

def Training1(dataset, trainSteps, batchSize, sparsePen, derivPen, eta, eta2, numInference, numTransforms):
    
    #intialize phi
    dim = dataset.shape[2]
    phi = np.random.rand(dim, dim ,numTransforms)
    
    #start training
    for trial in range(trainSteps):
        
        data_batch = dataset[np.random.randint(0,dataset.shape[0],batchSize)] #create random batch
        Input = data_batch[:,:9,:] #will predict next frame from these
        Output = data_batch[:,1:10,:] #the frames to be predicted
        
        #Find beta, fix, then optimize phi
        beta = inferBeta(Input, Output, phi, sparsePen, derivPen, eta2, numTransforms)
        phi = learnTransform(Input, Output, phi, beta, eta, Batchsize)
        
        #renormalize by capping absolute value to 1
        phi = phi/(np.abs(phi).max(dim=(0,1),keepdims=True))
        
    
    return(phi)
    

In [ ]:
def Localized_image(Input2, center, radius, epsilon):
    Local_matrix = np.empty(Input2.shape) #shape txn2, t for time, n2 for nxn image
    
    for t in range(Input2.shape[0]):
        for i in range(Input2.shape[1]):
            for j in range(Input2.shape[2]):
                Local_matrix[t,i,j] = np.abs(np.dot(np.array([i,j])-center[]))    

In [ ]:
def InferCenter(Input, Input2, center, radius, error, phi, beta, eta3, eta4, numInference2):
    
    #this same numInference parameter as with Inferbeta. Might be better to let update of ceneters/radii to have own number of steps
    
    for step in range(numInference2):
        attention = 
        localInput = 
        
    

In [4]:
def InferLocal(movie_seq, phi, sparsePen, derivPen, eta2): 
    #now want to infer beta, center, and radii for a single movie sequence
    #assume movie sequence is shape txn2, where t is length (time) and n2 is product of dimensions of 2d image
    

    dim = np.sqrt(movie_seq.shape[1])
    Input = movie_seq[:(time-1)]
    Output = movie_seq[1:]
    Input2 = Input.reshape((Input.shape[0],dim,dim)) #just reshape to make region of interest indices more natural
    numInference2 = 10 #this is arbitrary. want to update each of beta, c, r a little
    time = Input2.shape[0]
    
    beta = np.zeros((movie_seq.shape[0],phi.shape[2])) #initialize beta at 0. beta is shape tk. movie_seq first dim is t
    center = (dim/2)*np.ones((2,Input2.shape[0],phi.shape[2])) #initialize centers at center of image. each center is 2-vector
    radius = 17*np.ones((Input2.shape[0],phi.shape[2])) #initialize radius at 17, so start off region of interest as entire image
    